In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/ubuntu/Github/DataEngineerChallenge/spark-2.4.4-bin-hadoop2.7"

import numpy as np
import pandas as pd
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, max, sum, mean
from pyspark.sql.functions import col, when, count, countDistinct
from pyspark.sql.functions import split, concat_ws
from pyspark.sql.types import StructField, StructType
from pyspark.sql.types import StringType, FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import from_unixtime, unix_timestamp, to_date, trim




if "spark" not in dir():
    spark = SparkSession.builder \
    .appName("web_log_analysis") \
    .getOrCreate()
    
logFile = "data/2015_07_22_mktplace_shop_web_log_sample.log.gz"
numPartitions = 15
session_time = 15*60


def duration(start, end):
    try:
        num_of_seconds = (end - start).total_seconds()
    except:
        num_of_seconds = 0
    return num_of_seconds;

get_duration = udf(duration, FloatType())

def preprocess3(spark):
    #print(split("1.186.41.1",".").getItem(0).show());
    log_schema = StructType([
        StructField("timestamp", StringType(), False),
        StructField("elb", StringType(), False),
        StructField("client:port", StringType(), False),
        StructField("backend:port", StringType(), False),
        StructField("request_processing_time", StringType(), False),
        StructField("backend_processing_time", StringType(), False),
        StructField("response_processing_time", StringType(), False),
        StructField("elb_status_code", StringType(), False),
        StructField("backend_status_code", StringType(), False),
        StructField("received_bytes", StringType(), False),
        StructField("sent_bytes", StringType(), False),
        StructField("request", StringType(), False),
        StructField("user_agent", StringType(), False),
        StructField("ssl_cipher", StringType(), False),
        StructField("ssl_protocol", StringType(), False)])
    
    df = spark.read.csv(logFile, schema=log_schema, sep=" ").repartition(numPartitions).cache()
    split_client = split(df["client:port"], ":")
    split_backend = split(df["backend:port"], ":")
    split_request = split(df["request"], " ")

    df=df.withColumn("ip", split_client.getItem(0)) \
                .withColumn("client_port", split_client.getItem(1)) \
                .withColumn("backend_ip", split_backend.getItem(0)) \
                .withColumn("backend_port", split_backend.getItem(1)) \
                .withColumn("request_action", split_request.getItem(0)) \
                .withColumn("request_url", split_request.getItem(1)) \
                .withColumn("request_protocol", split_request.getItem(2)) \
                .withColumn("current_timestamp", col("timestamp").cast("timestamp")) \
                .drop("client:port","backend:port","request").cache()

    df=df.select(["ip", "request_url"]);
    
    
    df=df.na.drop(subset=["request_url"])
    df=df.na.drop(subset=["ip"])
    
    df = df.groupby("ip").agg(countDistinct("request_url").alias("count_unique_URLs"));
    df=df.na.drop(subset=["count_unique_URLs"])
    
    splitt2=split(df["ip"], "\\.");
    df=df.withColumn("octet0", splitt2.getItem(0));
    df=df.withColumn("octet1", splitt2.getItem(1));
    df=df.withColumn("octet2", splitt2.getItem(2));
    df=df.withColumn("octet3", splitt2.getItem(3));
    df=df.drop("ip");
    df=df.na.drop(subset=["octet0"])
    df=df.na.drop(subset=["octet1"])
    df=df.na.drop(subset=["octet2"])
    df=df.na.drop(subset=["octet3"])
    #print(df.dtypes);
    
    return df


def solve3(spark):
    dataset3 = preprocess3(spark).cache()
    dataset3.show();
    return dataset3.select("*").toPandas();

df=solve3(spark)

+-----------------+------+------+------+------+
|count_unique_URLs|octet0|octet1|octet2|octet3|
+-----------------+------+------+------+------+
|               16|    61|    16|   142|   162|
|               16|   117|   205|    39|   248|
|                2|   117|   203|   181|   144|
|               85|   115|   112|   250|   108|
|               34|   117|   241|   152|    20|
|               16|   202|   174|    92|    10|
|               10|   123|   136|   182|   137|
|              110|   202|    53|    89|   132|
|                6|    27|    63|   186|    72|
|              108|    14|   139|    82|   134|
|                3|   117|   207|    97|   173|
|              112|    27|    34|   244|   251|
|               84|   113|   193|   114|    25|
|               14|    59|   160|   110|   163|
|                7|   120|    61|    47|    36|
|                9|   117|   247|   188|    13|
|               88|   124|   125|    22|   218|
|               37|   103|    42|    88|

In [2]:
df=df[['octet0', 'octet1', 'octet2', 'octet3', 'count_unique_URLs']]

print(df.head());
print(df.shape)

  octet0 octet1 octet2 octet3  count_unique_URLs
0     61     16    142    162                 16
1    117    205     39    248                 16
2    117    203    181    144                  2
3    115    112    250    108                 85
4    117    241    152     20                 34
(90544, 5)


In [3]:
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

df=df.apply(pd.to_numeric) ;

X = df[['octet0', 'octet1', 'octet2', 'octet3']]
Y =df[['count_unique_URLs']]
model = xgboost.XGBRegressor(objective='reg:squarederror')
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X, Y, cv=kfold, scoring='neg_mean_squared_error')
#print(results);
print("RMSE:", np.mean(np.sqrt(np.abs(results))))

RMSE: 81.26309990779191
